# Applied Data Science Capstone Project

## Developed by:  Camilo Quiroga



In [1]:
import sys #!{sys.executable} -m pip install geocoder
import numpy as np
import pandas as pd
import geocoder
import requests 
from bs4 import BeautifulSoup

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
#Obtain data from url
url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(url.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'})
#print(soup.prettify()) 


In [3]:
#Preprocessing

postalCodes = []
boroughs= []
neighborhoods = []
columnNum = 1
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 3:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned' or cell.string == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
               

In [4]:
#Dataframe Construction

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighbors = pd.DataFrame(columns=column_names)

for data in range(0, len(postalCodes)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}
                                 , ignore_index=True)

neighbors.shape

(103, 3)

In [5]:
neighbors=neighbors.replace('\n','', regex=True)
neighbors=neighbors.replace('/',',', regex=True)
neighbors.sort_values('Borough',inplace=True)
neighbors.head()

,PostalCode,Borough,Neighborhood
86,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For..."
79,M4S,Central Toronto,Davisville
83,M4T,Central Toronto,"Moore Park , Summerhill East"
68,M5P,Central Toronto,Forest Hill North & West
74,M5R,Central Toronto,"The Annex , North Midtown , Yorkville"


In [6]:
neighbors.shape

(103, 3)